In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

#读取数据
train_df = np.load("./data/训练集/train_x.npy")
train_y = np.load("./data/训练集/train_y.npy")
test_df = np.load("./data/测试集A/test_x_A.npy")

In [ ]:
train_feat = pd.read_csv('./data/train_feat.csv')
test_feat = pd.read_csv('./data/test_feat.csv')

cao_train_feat = pd.read_excel('./cao/feature/feature_train.xlsx')
cao_test_feat = pd.read_excel('./cao/feature/feature_test.xlsx')
cao_train_feat.drop(columns='label',inplace=True)

cao_train_feat.columns = cao_train_feat.columns + '_cao'
cao_test_feat.columns = cao_test_feat.columns + '_cao'

train_feat['label'] = train_y

train_feat.shape,test_feat.shape 

In [ ]:
#样本平衡
train_df = train_feat[train_feat.label==0][:5000]._append(train_feat[train_feat.label!=0]).reset_index(drop=True)
train_y = train_df['label']
train_df = train_df.drop(columns='label')
test_df = test_feat.reset_index(drop=True)

#合并特征
train_df = pd.concat([train_df,cao_train_feat],axis=1)
test_df = pd.concat([test_df,cao_test_feat],axis=1)

In [ ]:
#对抗验证
import xgboost as xgb
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

#分离训练集，测试集

train_df['is_test'] = 0
test_df['is_test'] = 1
feat = [col for col in train_df if col not in ['label','is_test']]
print('筛选前 feat_num:',len(feat))


def Adversarial_val(train_df,test_df,feat_cols,lable):
    #合并
    df = pd.concat([train_df,test_df],axis=0,ignore_index=True)
    df = shuffle(df,random_state=16)
    # model = lgb.LGBMClassifier(metric='auc',verbose=-1).fit(df[feat_cols],df[lable])
    model = xgb.XGBClassifier(n_estimators=100,eval_metric= 'auc').fit(df[feat_cols],df[lable])
    #imp
    feat_imp_df = pd.DataFrame({'feature':feat_cols,'imp':np.zeros(len(feat_cols))})
    imp = model.feature_importances_
    feat_imp_df['imp'] = imp
    feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
    feat_imp_df['rank'] = range(feat_imp_df.shape[0])
    #预测
    df_pre = model.predict_proba(df[feat_cols])[:,1]
    auc_score = roc_auc_score(df['is_test'],df_pre)
    # print('AUC:',auc_score)
    return model,df_pre,feat_imp_df,auc_score

#特征筛选
select_feat = []
drop_feat = []

for feat in tqdm(feat):
    adv_model,df_pre,feat_imp_df,auc_score=Adversarial_val(train_df,test_df,[feat],'is_test')
    if auc_score<0.66:
        select_feat.append(feat)
    else:
        drop_feat.append(feat)
print('筛选后 feat_num:',len(select_feat))
feat_cols = select_feat

In [ ]:
# feat_cols = [col for col in train_df if col not in ['is_test','label','is_test_prob']]
hr_drop_cols = ['mean_nni_hr', 'sdnn_hr', 'sdsd_hr', 'nni_50_hr', 'pnni_50_hr',
       'nni_20_hr', 'pnni_20_hr', 'rmssd_hr', 'median_nni_hr', 'range_nni_hr',
       'cvsd_hr', 'cvnni_hr', 'mean_hr_hr', 'max_hr_hr', 'min_hr_hr',
       'std_hr_hr']
feat_cols =[col for col in select_feat if col not in ['bo_diff_1_mean', 'bo_diff_1_median', 'cvnni_bo', 'nni_50_hr', 'pnni_50_hr']+hr_drop_cols]
# feat_cols =select_feat
len(feat_cols)

In [ ]:
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
# 定义 X 和 y 变量
X = train_df[feat_cols]
y = train_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 使用 LazyRegressor 定义模型
multiple_ML_model = LazyClassifier(verbose=0,
          ignore_warnings=True,
          predictions=True)
# 对模型进行拟合，同时预测每个模型的输出结果
models, predictions = multiple_ML_model.fit(X_train, X_test, y_train, y_test)

In [ ]:
models

In [ ]:
y_test[:20]

In [ ]:
predictions[:20]

In [ ]:
#rf
import warnings
import lightgbm as lgb
# from lightgbm import early_stopping
# from lightgbm import log_evaluation
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,f1_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


def lgb_model(train_x, train_y, test_x):
    seeds=[2024]
    oof = np.zeros([train_x.shape[0], 3])
    test_predict = np.zeros([test_x.shape[0], 3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    for seed in seeds:
        print('Seed:',seed)
        folds = 10
        kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        acc_scores = []
        # train_x = train_x.values
        # train_y = train_y.values
        
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            print("|  Model  Fold  {}  Training Start           |".format(str(i + 1)))
            
            trn_x, trn_y, val_x, val_y = train_x.values[train_index], train_y[train_index], train_x.values[valid_index], \
                                        train_y[valid_index] 
            
            params={'n_estimators':500}
            model = RandomForestClassifier(**params).fit(trn_x, trn_y)

            val_pred  = model.predict_proba(val_x)
            test_pred = model.predict_proba(test_x)

            #特征重要性
            # feat_imp_df['imp'] += lgb_model.feature_importance(importance_type='gain') / folds/ len(seeds)
            # feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
            # feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            
            oof[valid_index] = val_pred/ len(seeds)
            test_predict += test_pred / kf.n_splits / len(seeds)
            
            acc_score = accuracy_score(val_y,np.argmax(val_pred,axis=1))
            print(acc_score)
            acc_scores.append(acc_score)
            print('AVG_acc :',sum(acc_scores)/len(acc_scores))
            print('std :',np.std(acc_scores))
        print('oof:',classification_report(train_y,np.argmax(oof,axis=1)))
        
    return oof, test_predict,feat_imp_df

# 训练LGB模型
rf_oof, rf_test, lgb_imp_df = lgb_model(train_df[feat_cols].fillna(0), train_y, test_df[feat_cols].fillna(0))

In [33]:
#ExtraTreesClassifier
import warnings
import lightgbm as lgb
# from lightgbm import early_stopping
# from lightgbm import log_evaluation
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,f1_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


def lgb_model(train_x, train_y, test_x):
    seeds=[2024]
    oof = np.zeros([train_x.shape[0], 3])
    test_predict = np.zeros([test_x.shape[0], 3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    for seed in seeds:
        print('Seed:',seed)
        folds = 10
        kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        acc_scores = []
        # train_x = train_x.values
        # train_y = train_y.values
        
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            print("|  Model  Fold  {}  Training Start           |".format(str(i + 1)))
            
            trn_x, trn_y, val_x, val_y = train_x.values[train_index], train_y[train_index], train_x.values[valid_index], \
                                        train_y[valid_index] 
            
            params={'class_weight':'balanced', 'max_depth':25,'min_samples_leaf':50,'n_estimators':500, 'random_state':2024}
            model = ExtraTreesClassifier().fit(trn_x, trn_y)

            val_pred  = model.predict_proba(val_x)
            test_pred = model.predict_proba(test_x)

            #特征重要性
            # feat_imp_df['imp'] += lgb_model.feature_importance(importance_type='gain') / folds/ len(seeds)
            # feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
            # feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            
            oof[valid_index] = val_pred/ len(seeds)
            test_predict += test_pred / kf.n_splits / len(seeds)
            
            acc_score = accuracy_score(val_y,np.argmax(val_pred,axis=1))
            print(acc_score)
            acc_scores.append(acc_score)
            print('AVG_acc :',sum(acc_scores)/len(acc_scores))
            print('std :',np.std(acc_scores))
        print('oof:',classification_report(train_y,np.argmax(oof,axis=1)))
        
    return oof, test_predict,feat_imp_df

# 训练ExtraTreesClassifier模型
etc_oof, etc_test, lgb_imp_df = lgb_model(train_df[feat_cols].fillna(0), train_y, test_df[feat_cols].fillna(0))

Seed: 2024
|  Model  Fold  1  Training Start           |
0.6894117647058824
AVG_acc : 0.6894117647058824
std : 0.0
|  Model  Fold  2  Training Start           |
0.7307692307692307
AVG_acc : 0.7100904977375566
std : 0.020678733031674168
|  Model  Fold  3  Training Start           |
0.7182103610675039
AVG_acc : 0.712797118847539
std : 0.017312565634903458
|  Model  Fold  4  Training Start           |
0.7299843014128728
AVG_acc : 0.7170939144888725
std : 0.01673860970921052
|  Model  Fold  5  Training Start           |
0.728414442700157
AVG_acc : 0.7193580201311294
std : 0.015641276847975664
|  Model  Fold  6  Training Start           |
0.7362637362637363
AVG_acc : 0.7221756394865638
std : 0.015606713684280613
|  Model  Fold  7  Training Start           |
0.706436420722135
AVG_acc : 0.719927179663074
std : 0.015463094674594386
|  Model  Fold  8  Training Start           |
0.7095761381475667
AVG_acc : 0.7186332994736355
std : 0.014863975457911115
|  Model  Fold  9  Training Start          

In [ ]:
#xgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def xgb_model(train_x, train_y, test_x):
    seeds=[618]
    oof = np.zeros([train_x.shape[0], 3])
    test_predict = np.zeros([test_x.shape[0], 3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    for seed in seeds:
        print('Seed:',seed)
        folds = 5
        kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        acc_scores = []
        # train_x = train_x.values
        # train_y = train_y.values
        
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            print("|  Model  Fold  {}  Training Start           |".format(str(i + 1)))
            
            trn_x, trn_y, val_x, val_y = train_x.values[train_index], train_y[train_index], train_x.values[valid_index], \
                                        train_y[valid_index] 
            
            xgb_params = {
                'booster': 'gbtree','objective': 'multi_class','n_estimators':20000,'max_depth': 8,
                'lambda': 12,'subsample': 0.7,'num_class':3,'colsample_bytree': 0.8,'colsample_bylevel': 0.7,
                'eta': 0.1,'tree_method': 'hist','seed': seed,'nthread': 16,
                }
            
            #训练模型
            xgb_model = xgb.XGBClassifier(**xgb_params)
            xgb_model.fit(trn_x,trn_y,eval_set=[(trn_x, trn_y),(val_x,val_y)],early_stopping_rounds=100,verbose=1000)
            
            val_pred  = xgb_model.predict_proba(val_x)
            test_pred = xgb_model.predict_proba(test_x)
            feat_imp_df['imp'] += xgb_model.feature_importances_ / folds/ len(seeds)
            feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
            feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            
            oof[valid_index] = val_pred / len(seeds)
            test_predict += test_pred / kf.n_splits / len(seeds)
            
            acc_score = accuracy_score(val_y,np.argmax(val_pred,axis=1))
            print(acc_score)
            acc_scores.append(acc_score)
            print('AVG_acc :',sum(acc_scores)/len(acc_scores))
        print('oof:',classification_report(train_y,np.argmax(oof,axis=1)))
        feat_imp_df.head(25)
        
    return oof, test_predict,feat_imp_df

# 训练 XGB模型
xgb_oof, xgb_test, xgb_imp_df = xgb_model(train_df[feat_cols], train_y, test_df[feat_cols])

In [ ]:
#lgb
import warnings
import lightgbm as lgb
# from lightgbm import early_stopping
# from lightgbm import log_evaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,f1_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


def lgb_model(train_x, train_y, test_x):
    seeds=[2024]
    oof = np.zeros([train_x.shape[0], 3])
    test_predict = np.zeros([test_x.shape[0], 3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    for seed in seeds:
        print('Seed:',seed)
        folds = 12
        kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        acc_scores = []
        # train_x = train_x.values
        # train_y = train_y.values
        
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            print("|  Model  Fold  {}  Training Start           |".format(str(i + 1)))
            
            trn_x, trn_y, val_x, val_y = train_x.values[train_index], train_y[train_index], train_x.values[valid_index], \
                                        train_y[valid_index] 
            
            lgb_params={"boosting_type": "gbdt","objective": "multiclass","metric": "multi_logloss",
                        'num_class':3,"max_depth": -1,"learning_rate": 0.1,"colsample_bytree": 0.7,
                        "colsample_bynode": 0.8,"verbose": -1,"random_state": seed,"extra_trees":True,
                        'num_leaves':128,"verbose": -1,"max_bin":128}
            dtrain = lgb.Dataset(trn_x, label=trn_y)
            dval = lgb.Dataset(val_x, label=val_y)
            lgb_model = lgb.train(lgb_params, dtrain, valid_sets=[dval], num_boost_round=20000,callbacks=[lgb.early_stopping(100), lgb.log_evaluation(500)])

            val_pred  = lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration)
            test_pred = lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration)

            #特征重要性
            feat_imp_df['imp'] += lgb_model.feature_importance(importance_type='gain') / folds/ len(seeds)
            feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
            feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            
            oof[valid_index] = val_pred/ len(seeds)
            test_predict += test_pred / kf.n_splits / len(seeds)
            
            acc_score = accuracy_score(val_y,np.argmax(val_pred,axis=1))
            print(acc_score)
            acc_scores.append(acc_score)
            print('AVG_acc :',sum(acc_scores)/len(acc_scores))
            print('std :',np.std(acc_scores))
        print('oof:',classification_report(train_y,np.argmax(oof,axis=1)))
        
    return oof, test_predict,feat_imp_df

# 训练LGB模型
lgb_oof, lgb_test, lgb_imp_df = lgb_model(train_df[feat_cols], train_y, test_df[feat_cols])

In [ ]:
#加权融合
import matplotlib.pyplot as plt

scores = []; Weights = []
best_m = 0
best_f1=0
for m in np.arange(0.01,1,0.001):
    f1 = f1_score(train_y,np.argmax((m*lgb_oof+(1-m)*xgb_oof),axis=1),average='micro')
    scores.append(f1)
    Weights.append(m)
    if f1 > best_f1:
        best_f1 = f1
        best_m = m
# PLOT
plt.figure(figsize=(20,5))
plt.plot(Weights,scores,'-o',color='blue')
plt.scatter([best_m], [best_f1], color='blue', s=300, alpha=1)
plt.xlabel('Threshold',size=14)
plt.ylabel('Validation F1 Score',size=14)
plt.title(f'Best score = {best_f1:.3f} at Best Weights = {best_m:.3}',size=18)
plt.show()

In [35]:
etc_test

array([[0.242, 0.153, 0.605],
       [0.04 , 0.3  , 0.66 ],
       [0.023, 0.387, 0.59 ],
       ...,
       [0.867, 0.035, 0.098],
       [0.586, 0.254, 0.16 ],
       [0.303, 0.432, 0.265]])

In [34]:
submission = pd.read_csv('./sub/submit_example_A.csv')
submission['label'] = pd.DataFrame(np.argmax(etc_test,axis=1))
submission.to_csv('./sub/result.csv', index = False)